# Базовое решение для задачи A 

In [ ]:
import pandas as pd
import numpy as np

from sklearn.ensemble import GradientBoostingClassifier

Читаем входные файлы с данными

In [ ]:
transactions = pd.read_csv('./transactions.csv')
customers_gender = pd.read_csv('./customers_gender_train.csv')

Для каждого клиента подсчитываем различные mcc_code

In [ ]:
X = transactions.groupby('customer_id')[['mcc_code']] \
                    .apply(lambda x: x.unstack().value_counts()) \
                    .unstack() \
                    .fillna(0)

In [ ]:
customers_gender = customers_gender.set_index('customer_id')

In [ ]:
Y_train = customers_gender.loc[X.index].gender
Y_train = Y_train.reset_index()
del Y_train['customer_id']
Y_train = Y_train.dropna(0)

In [ ]:
X_train = X.reset_index()
X_train = X_train.loc[Y_train.index].set_index('customer_id')

Обучаемся на всех данных

In [ ]:
clf = GradientBoostingClassifier(random_state=13)
clf.fit(X_train, Y_train)

Предсказываем пол для тестовых данных и создаём файл с ответом

In [ ]:
X_test = X.drop(customers_gender.index)
result = pd.DataFrame(X_test.index, columns=['customer_id'])
result['gender'] = clf.predict_proba(X_test)[:, 1]

In [ ]:
result.to_csv('baseline_a.csv', index=False)